In [1]:
import minio
from minio.error import S3Error
from minio.commonconfig import ENABLED
from minio.versioningconfig import VersioningConfig

import pathlib
import pandas as pd
import os
import json
from tqdm import tqdm

from minio_utils import * 
#####-------------------------------------------------------------#####
# MAIN RUN
#####-------------------------------------------------------------#####
# Create a new bucket
minio_credentials = "credentials.mb.json"
create_bucket(bucket_name = "dev", minio_credentials = minio_credentials)



Bucket 'dev' created successfully.


True